# U.T. 2. Aprendizaje supervisado (I).
# Introducción

No existe ningún algoritmo que se ajuste a todos los posibles escenarios.
Hay que probar varios algoritmos y comparar el rendimiento entre ellos para elegir el que mejor se adapte.
Hay que usar métricas semejantes en todos los algoritmos.

La librería scikit-learn ofrece una gran variedad de algoritmos de aprendizaje ya implementados junto con cantidad de
funciones para prepocesar los datos, optimizar los algoritmos y evaluar los modelos

El procedimiento es el siguiente:
1. Elegir las características y recoger los ejemplos etiquetados.
2. Procesar los datos y dividirlos en grupos.
3. Elegir la métrica.
4. Elegir un algoritmo de clasificación y optimización.
5. Evaluar el rendimiento del modelo.
6. Optimizar el algoritmo cambiando los hiperparámetros, pasar al punto 5 hasta que sea óptimo

## Perceptron
### Preparación de los datos
Los pasos necesarios para la preparación de los datos son:
1. Elección de los datos
2. Conversión de las características categóricas en valores numéricos
3. Dividir el conjunto de datos de entrada en datos de entrenamiento y datos de test
    - Se divide en dos conjuntos los datos, unos se utilizará para entrenar el algoritmo.
    - El segundo se guarda para las métricas.
    - El segundo tendrá un tamaño entre 25% y 30%.
    - La división se hará semejante por clases (y).

`X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)`

![](img/ut02_00.png)

#### train_test_split
`train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)`

Divide dos conjuntos de datos según las características que indiquemos
- **test_size** indica el porcentaje del segundo grupo de datos.
- **stratify=y** indica que cada conjunto tenga un porcentaje similar según las clases o etiquetas (y).
- **random_state** se utiliza para poder reproducir resultados, es el valor de inicialización de generador aleatorio.
- Esta función mezcla los datos al hacer los grupos, por lo que no tendremos que hacerlo nosotros.

***Es importante recordar que los ejemplos se deben suministrar de forma aleatoria si queremos mejorar la convergencia***

### Estandarización
El orden de magnitud de las variables influye en los procedimientos estadísticos.

Por ejemplo, si tenemos una muestra donde se recogen el peso y la altura de ciertos individuos es importante las
unidades en cómo se miden las variables. Si la altura la cuantificamos en metros, los individuos posiblemente estarían
en el intervalo [0; 2] mientras que el peso, si fuese medido en kilogramos, en el intervalo [0; 200].
Una variable tiene un rango 100 veces más grande que la otra.

La estandarización:
- Mejora la convergencia.
- No es imprescindible pero hay que probarla.
- Se debe realizar con los datos de entrenamiento y aplicar a los demás datos.

<code>
sc = StandardScaler()<br>
sc.fit(X_train)<br>
X_train_std = sc.transform(X_train)<br>
X_test_std = sc.transform(X_test)<br>
</code>

### Entrenamiento y métrica
Las métricas se usan para comparar rendimientos.
Existen gran cantidad de métricas.
El algoritmo incorpora métricas propias.
Existen métricas externas.
La métrica nos puede dar el error producido o la exactitud del mismo.
El uso en la métrica del error producido frente a la exactitud del algoritmo es cuestión de gustos, ya que ambos
están relacionados según: exactitud = 1 – error.

<code>
ppn = Perceptron(eta0=0.1, random_state=1)<br>
ppn.fit(X_train_std, y_train)<br>
print("Tasa:", ppn.score(X_test_std, y_test))<br>
y_pred = ppn.predict(X_test_std)<br>
print("Tasa:", accuracy_score(y_test, y_pred))<br>
</code>


In [1]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print("Valores de las etiquetas", np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=1, stratify=y)
print("Número de etiquetas por casos:", np.bincount(y),
                       np.bincount(y_train),np.bincount(y_test))
# escalado: Normalización. Recordar que el ajuste se hace sobre los datos de
#       entrenamiento y la transformación se aplica a ambos, prueba y entrenamiento
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

ppn = Perceptron(eta0=0.1, random_state=1)
ppn.fit(X_train_std, y_train)
print("Tasa:", ppn.score(X_test_std, y_test))

# predecir datos y comparar con los originales
y_pred = ppn.predict(X_test_std)
print("Tasa:", accuracy_score(y_test, y_pred))

Valores de las etiquetas [0 1 2]
Número de etiquetas por casos: [50 50 50] [35 35 35] [15 15 15]
Tasa: 0.9777777777777777
Tasa: 0.9777777777777777


![](img/ut02_01.png)

**Probar a no estandarizar los datos**

![](img/ut02_02.png)
![](img/ut02_03.png)


### Sobreajuste
Uno de los problemas principales es el **sobreajuste**.
Los datos se ajustan muy bien a los datos de entrenamiento pero no a los de test


## Regresión logística
Es uno de los métodos más utilizados en la industria, es un algoritmo de clasificación aunque su nombre indica regresión.
Es un modelo de clasificación binaria que se puede generalizar para múltiples clases: Regresión logística multinomial o
Softmax.

Conceptos matemáticos
- Se usa la función sigmoide como función de activación.
- La función sigmoide recoge un entero y lo transforma en real, y= 0,5 Cuando x=0.
- El resultado es la probabilidad de que un ejemplo pertenezca a una clase concreta.
- Es muy importante el valor de la certeza: cuánto estamos seguros de que la predicción sea correcta.
- Hay que minimizar la función sigmoide para usar el descenso de gradiente estocástico (SGD).

In [2]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print("Valores de las etiquetas", np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                                 random_state=1, stratify=y)
print("Número de etiquetas por casos:", np.bincount(y),
                       np.bincount(y_train),np.bincount(y_test))
# escalado: Normalización. Recordar que el ajuste se hace sobre los datos de
#       entrenamiento y la transformación
#  se aplica a ambos, prueba y entrenamiento
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

lrn = LogisticRegression(C=100, random_state=1, solver='lbfgs', multi_class='ovr')
# C valores menores mayor ajuste, se verá más adelante el concepto
# solver = liblinear para clasifiación binaria y lbfgs para multiclase
# multi_class = ovr hace que se utilice ese algoritmo para la división entre clases
#    asigna 1 a la clase positiva y al resto 0, hace tantas características nuevas como
#    necesite
lrn.fit(X_train_std, y_train)
print("Tasa:", lrn.score(X_test_std, y_test))
y_pred = lrn.predict(X_test_std)
print("Tasa:", accuracy_score(y_test, y_pred))

# predecimos ahora los tres primeros
print(lrn.predict_proba(X_test_std[:3, ]))
# [[3.81527885e-09 1.44792866e-01 8.55207131e-01]     85% en la clase tercera
#  [8.34020679e-01 1.65979321e-01 3.25737138e-13]     83% en la primera clase
#  [8.48831425e-01 1.51168575e-01 2.62277619e-14]]    84% en la primera clase
# Clases predichas:
print("Clases de las tres primeras filas:", lrn.predict(X_test_std[:3, :]))  # [2 0 0]
print("Clase de la fila 3:", lrn.predict(X_test_std[2:3, :]))  # [0]


Valores de las etiquetas [0 1 2]
Número de etiquetas por casos: [50 50 50] [35 35 35] [15 15 15]
Tasa: 0.9777777777777777
Tasa: 0.9777777777777777
[[3.81527885e-09 1.44792866e-01 8.55207131e-01]
 [8.34020679e-01 1.65979321e-01 3.25737138e-13]
 [8.48831425e-01 1.51168575e-01 2.62277619e-14]]
Clases de las tres primeras filas: [2 0 0]
Clase de la fila 3: [0]


![](img/ut02_04.png)

### Parámetros de la regresión logística
- **C** (float), default=1.0. Es la inversa de la regularización, debe ser un valor real positivo. Valores más pequeños implican mayor regularización
- **solver**{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’. Algoritmo a usar para realizar el ajuste.
    - Para pequeños dataset y problemas binarios se utiliza liblinear si el dataset es grande mejor usar ‘sag’ y ‘saga’ que son más rápidos
    - Para problemas con más de dos clases se tiene que usar ‘newton-cg’, ‘sag’, ‘saga’ y ‘lbfgs’
    - Los algoritmos ‘newton-cg’, ‘lbfgs’, ‘sag’ y ‘saga’  pueden usarse con la regularización L2
    - Los algoritmos ‘liblinear’ y ‘saga’ permiten también a regularización L1
    - Los algoritmos ‘sag’ y ‘saga’ convergerán solo si las características están aproximadamente en la misma escala.
- **multi_class**{‘auto’, ‘ovr’, ‘multinomial’}, default=’auto’. La opción ovr se debe usar para problemas de multiclass en los que cada característica es binaria. La opción multinomial no está disponible para el solver liblinear
- **n_jobs** int, default=None. Número de CPUs a usar de forma paralela. Se usará solo si se elige multi_class=’ovr’, es ignorado para ‘liblinear’. -1 significa usar todos los procesadores posibles.
- **penalty**{‘l1’, ‘l2’, ‘elasticnet’, ‘none’}, default=’l2’. Usado para determinar la regularización a usar.



## Regularización
Generalmente se pretende reducir la capacidad del modelo y / o reducción de la varianza de las predicciones para mejorar
la convergencia

La regularización puede ser entendida como el proceso de agregar información (cambiar la función objetivo) para evitar
el sobreajuste

### Qué es el sobreajuste
Problema común en el ML.
Significa que el modelo se ajusta muy bien a los datos de entrenamiento, pero no también a datos nuevos (test).
Solución: reducir la complejidad del modelo: por ejemplo regularizando, hay más opciones.

![](img/ut02_05.png)

- En azul la curva de ajuste para los datos de prueba
- En naranja la misma curva para los datos de test
- Eje Y el error, eje X número de épocas
- Podemos ver cómo el error no cambia mucho en los datos de test (val_loss)
- Vemos que la diferencia se amplía


### Qué es el subajuste
Problema común en el ML.
El modelo no es lo suficiente complejo para capturar la estructura de los datos
Solución: aumentar la complejidad del modelo.

![](img/ut02_06.png)

- En azul la curva de ajuste para los datos de prueba
- En naranja la misma curva para los datos de test
- Eje Y el error, eje X número de épocas
- Podemos ver cómo el error no cambia mucho en los datos de test (val_loss) pero permanece constante

### Sin errores
- Azul datos de prueba, naranja datos de test
- Una gráfica es error, la otra es rendimiento, ver cómo son complementarias

![](img/ut02_07.png)

### Equilibrio tendencia-varianza (bias-variance)
Se utiliza para describir el rendimiento de un modelo.

Una alta varianza es proporcional a un sobreajuste. La varianza mide la consistencia del modelo y es sensible al orden
de los datos de entrada.

La tendencia mide cómo de lejos estás las predicciones de la realidad y no es susceptible al orden o a diferentes
conjuntos de entrenamiento.

No se puede conseguir la mínima varianza y la mínima tendencia, están relacionadas, hay que llegar a un equilibrio.

## Máquinas vector soporte
Pueden ser consideradas una extensión del perceptron.
Se basan en minimizar los errores de clasificación maximizando el margen. El **margen** se define como la distancia
entre el hiperplano de separación (bordes de decisión) y los ejemplos que están más cerca al mismo que se denominan:
vectores de soporte.
La idea es maximizar el margen, hacerlo lo más grande posible.

![](img/ut02_08.jpg)

El parámetro C se comporta como un control de la penalización. En concreto valores pequeños de C dan valores grandes
de penalización en la clasificación de los errores y viceversa.

En definitiva, se puede controlar la anchura del margen con el parámetro C. C pequeño, margen grande y viceversa.

El parámetro C lo hemos visto anteriormente: LogisticRegression(C=100…

![](img/ut02_09.png)

![](img/ut02_10.png)

In [5]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
print("Valores de las etiquetas", np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
print("Número de etiquetas por casos:", np.bincount(y), np.bincount(y_train), np.bincount(y_test))

# escalado: Normalización. Recordar que el ajuste se hace sobre los datos de entrenamiento y la transformación
#  se aplica a ambos, prueba y entrenamiento
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)


svm = SVC(kernel='linear', C=1.0, random_state=1, probability=True)
svm.fit(X_train_std, y_train)

print("Tasa:", svm.score(X_test_std, y_test))
y_pred = svm.predict(X_test_std)
print("Tasa:", accuracy_score(y_test, y_pred))

# predecimos ahora los tres primeros
print(svm.predict_proba(X_test_std[:3, ]))
# [[0.00772374 0.01640841 0.97586785]     97%
#  [0.92249209 0.05497364 0.02253427]     92%
#  [0.95813714 0.02701406 0.0148488 ]]     95%
# Clases predichas:
print("Clases de las tres primeras filas:", svm.predict(X_test_std[:3, :]))  # [2 0 0]
print("Clase de la fila 3:", svm.predict(X_test_std[2:3, :]))  # [0]

Valores de las etiquetas [0 1 2]
Número de etiquetas por casos: [50 50 50] [35 35 35] [15 15 15]
Tasa: 0.9777777777777777
Tasa: 0.9777777777777777
[[0.00772374 0.01640841 0.97586785]
 [0.92249209 0.05497364 0.02253427]
 [0.95813714 0.02701406 0.0148488 ]]
Clases de las tres primeras filas: [2 0 0]
Clase de la fila 3: [0]


![](img/ut02_11.png)


### Parmámetros
- **C** (float), default=1.0. Es la inversa de la regularización, debe ser un valor real positivo. Valores más pequeños implican mayor regularización y margen más grande (recordar que se busca maximizar el margen).
- **kernel**{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’. Algoritmo a usar para realizar el ajuste.
- **degree** int, default=3. Coeficiente del polinomio a usar si el kernel es poly
- **coef0** float, default=0.0. Valor del término independiente si el kernel es poly
- **probability** bool, default=False. Indica si hay que estimar la probabilidad de una clase
- **gamma**{‘scale’, ‘auto’} or float, default=’scale’. Coeficiente gamma para rbf, poly y sigmoid

### Problemas no lineales

La razón de que MVS sea tan popular es por la posibilidad de usar diferentes kernels para solucionar los problemas
de clasificación que no son linealmente separables.

La idea principal es crear combinaciones no lineales de las características originales y proyectarlas a un espacio
dimensional mayor que sea separable linealmente.

El problema de construir esas características es que son computacionalmente muy costosas.

![](img/ut02_12.png)

#### Problemas no lineales: rbf
Uno de los kernels más usado es el RBF (Radial basic function) o kernel Gausiano. γ (gamma) es un parámetro libre
para optimizar. Establecido a un valor pequeño hará que los bordes del área sean más suaves,
mientras que un valor mayor los convertirá en más abruptos.

<code>
svc = SVC(kernel=’rbf’, random_state=1, gamma=0.2, C=1.0)<br>
svc = SVC(kernel=’rbf’, random_state=1, gamma=100, C=1.0)<br>
</code>

#### Problemas no lineales: polinomial
Otra aproximación es añadir más características en forma de características polinomiales como combinación lineal de
 las características iniciales

<code>
svc = SVC(kernel=’poly’, degree=3, coef0=1, C=1.0)<br>
</code>

## Árboles de decisión
Un árbol de decisión es un algoritmo que divide los datos en función de una serie de preguntas.
Se empieza desde la raíz y se va dividiendo los datos de las características buscando la mayor ganancia de información (Ig).
Este proceso se hace de forma iterativa durante la división de los datos. Si no ponemos límites a la iteración llegaremos
al sobreajuste.

![](img/ut02_13.png)

### Algoritmos para el parámetro cryterion
Se busca maximizar la función objetivo, hay tres algoritmos
- **Entropía**. Esta función es máxima cuando la distribución es uniforme de los ejemplos y cero cuando todos los
ejemplos de un nodo son de la misma clase.
- **Gini**. Se puede entender como un criterio para minimizar la probabilidad de errores mal clasificados.
Es máxima si las clases están perfectamente mezcladas.
- **Error de clasificación**. Se utiliza sobre todo para realzar la poda del árbol en vez de para su creación.

De los tres criterios anteriores, los dos primeros funcionan de forma similar.
Generalmente el mecanismo Gini está en medio entre la Entropía y el Error de clasificación.
No es necesario escalar los datos para este mecanismo.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=1)

### Parámetros
- **criterion**{"gini”, "entropy”}, default=”gini”. Algoritmo a usar
- **splitter**{"best”, "random”}, default=”best”. Estrategía para divider cada nodo.
- **max_depth** int, default=None. Máxima profundidad del árbol, si es muy grando aparecerá el sobreajuste
- **min_samples_split** int or float, default=2. Número mínimo de ejemplos para divider un nodo.
- **min_samples_leaf** int or float, default=1. Mínimo número de ejemplos que tiene que haber en un nodo hoja

![](img/ut02_14.png)

## Bosques aleatorios
Un bosque utiliza un conjunto de árboles como base para realizar las decisiones y separar los ejemplos.
El parámetro más importante es el número de árboles (K).
La mayor ventaja es que no hay que preocuparse por los hiperparámetros. En general, Cuantos más árboles más rendimiento
en el clasificador, pero más coste computacional.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='gini', n_estimators= 25, random_state=1, n_jobs=-1)

![](img/ut02_15.png)

### Parámetros
- **n_estimators** int, default=100. Número de árboles a usar
- **criterion**{"gini”, "entropy”}, default=”gini”. Algoritmo a us
- **splitter**{"best”, "random”}, default=”best”. Estrategía para divider cada nodo.
- **max_depth** int, default=None. Máxima profundidad del árbol, si es muy grando aparecerá el sobreajuste
- **min_samples_split** int or float, default=2. Número mínimo de ejemplos para divider un nodo.
- **min_samples_leaf** int or float, default=1. Mínimo número de ejemplos que tiene que haber en un nodo hoja

## K-Vecinos
Este algoritmo hace conjuntos de k-vecinos basándose en la distancia que nosotros elijamos.
Se adapta muy bien a recoger datos de prueba on-line, pero el coste computacional se incrementará de forma lineal con
el número de nuevos casos.
La correcta elección del parámetro k (número de vecinos n_neighbors en el código anterior) es crucial para un buen
funcionamiento, así como de la métrica.
Generalmente la distancia euclídea es una buena aproximación, pero deberemos estandarizar los datos

![](img/ut02_16.png)

### Algoritmo
1. Elegir el número de k vecinos y la métrica de distancia.
2. Encontrar los k-vecinos más cercanos en función de la métrica.
3. Asignar la clase en función del voto mayoritario.

Existen dos parámetros claves, el número de vecinos (k) y la métrica. Si la métrica incluye algún otro parámetro
también será necesario establecerlo.

### Métricas
|Identificador|Nombre de clase|args|Función de distancia|
|-------------|---------------|----|--------------------|
|"euclidean”|EuclideanDistance|-|sqrt(sum((x - y)^2))|
|"manhattan”|ManhattanDistance|-|sum(|x - y|)|
|"chebyshev”|ChebyshevDistance|-|max(|x - y|)|
|"minkowski”|MinkowskiDistance|p|sum(|x - y|^p)^(1/p)|
|"wminkowski”|WMinkowskiDistance|p, w|sum(|w * (x - y)|^p)^(1/p)|
|"seuclidean”|SEuclideanDistance|V|sqrt(sum((x - y)^2 / V))|
|"mahalanobis”|MahalanobisDistance|V or VI|sqrt((x - y)' V^-1 (x - y))|

### Problema
Si los vecinos están muy separados entre sí en un espacio de alta dimensionalidad.
En este caso se recurren a las técnicas de reducción de dimensionalidad o a la reducción de características (PCA).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')

### Parámetros
- **n_nvighbors** int, default=5. Número de vecinos a usar por grupo.
- **p** int, default=2. Potencia de la métrica minkowski.
- **metric** str. Cadena con el nombre de la métrica.